# Card Abstraction Visualization
Background: I first went through `abstraction_exploration.ipynb` to better understand how many clusters I should be using for K-Means clustering, concluding that 100 clusters per game stage was a good number using the elbow method.

Using `abstraction.py`, I then randomly generated 100,000 hands per game stage (flop/turn/river), and then generated the clusters on this data using K-Means Clustering using the euclidean distance, which are found in the `data/clusters` folder. 

Now, in this notebook, I attempt to visualize the clusters generated.

In [1]:
import joblib
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

# Flop Visualization

In [2]:
flop_kmeans_centroids = joblib.load('../data/clusters/flop/1669110120_samples=100000_bins=5.npy')
flop_raw_data = np.load('../data/raw/flop/1669110120_samples=100000_bins=5.npy')
flop_cards = joblib.load('../data/raw/flop/1669110120_samples=100000_bins=5')

In [3]:
# Convert histograms to EHS
ehs_flop_raw_data = (flop_raw_data * np.array([0.1, 0.3, 0.5, 0.7, 0.9])).sum(axis=1)

In [4]:
assert(flop_kmeans_centroids.shape[0] == 100) # This should be 100
kmeans = KMeans(flop_kmeans_centroids.shape[0])
kmeans.cluster_centers_ = flop_kmeans_centroids
kmeans._n_threads = -1

flop_raw_data_classes = kmeans.predict(flop_raw_data)
flop_raw_data_classes

array([51, 62, 19, ..., 63, 24, 29], dtype=int32)

In [5]:
flop_kmeans_centroids.shape

(100, 5)

The raw data is 5 dimensional, so we first run PCA to reduce this data down to 3 dimensions.

In [6]:
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [7]:
pca_3d = PCA(n_components=3) # 3D vizualization

In [8]:
PCs_3d = pca_3d.fit_transform(flop_raw_data)
PCs_3d_centroids = pca_3d.fit_transform(flop_kmeans_centroids)

In [9]:
df = pd.DataFrame(PCs_3d, columns=['dim1', 'dim2', 'dim3'])
df_centoids = pd.DataFrame(PCs_3d_centroids, columns=['dim1', 'dim2', 'dim3'])


In [10]:
marker_text = []
for i, card_string in enumerate(flop_cards):
	cards = card_string.split(' ')
	assert(len(cards) == 5)
	marker_text.append(f'Player Cards: {cards[:2]}, Community Cards: {cards[2:]}, Class: {flop_raw_data_classes[i]}, EHS: {ehs_flop_raw_data[i]:.2f}')
marker_text[:5]

["Player Cards: ['Kh', 'Kd'], Community Cards: ['6d', '8h', '9s'], Class: 51, EHS: 0.75",
 "Player Cards: ['8s', 'Kh'], Community Cards: ['5s', '4d', '5c'], Class: 62, EHS: 0.48",
 "Player Cards: ['Jh', '2c'], Community Cards: ['Js', 'Kh', '7s'], Class: 19, EHS: 0.71",
 "Player Cards: ['8c', '7s'], Community Cards: ['3d', '3c', 'Ah'], Class: 75, EHS: 0.25",
 "Player Cards: ['8s', 'Tc'], Community Cards: ['Ad', 'Th', 'Ah'], Class: 28, EHS: 0.74"]

In [11]:
""" TODO:
- Add Title
- Animate rotation https://community.plotly.com/t/how-to-animate-a-rotation-of-a-3d-plot/20974
"""

' TODO:\n- Add Title\n- Animate rotation https://community.plotly.com/t/how-to-animate-a-rotation-of-a-3d-plot/20974\n'

In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=PCs_3d[:, 0], y=PCs_3d[:,1], z=PCs_3d[:,2],
                        hovertemplate='<b>%{text}</b><extra></extra>',
                        text=marker_text,
                        marker=dict(
                            size=2,
                            color=flop_raw_data_classes
                        ),
                        mode='markers',
))
fig.update_layout(title=dict(text=f"Flop Abstraction ({flop_kmeans_centroids.shape[0]} Clusters)", font=dict(size=28)), title_x=0.5, scene=dict(
                  xaxis_title="Dimension 1",
                  yaxis_title="Dimension 2",
                  zaxis_title="Dimension 3")
)
# fig.show()
fig.show(renderer='browser')

# Turn Visualization
Same steps as for flop.

In [39]:
kmeans_centroids = joblib.load('../data/clusters/turn/1669110257_samples=100000_bins=5.npy')
raw_data = np.load('../data/raw/turn/1669110257_samples=100000_bins=5.npy')
cards = joblib.load('../data/raw/turn/1669110257_samples=100000_bins=5')

# Convert histograms to EHS
ehs_raw_data = (raw_data * np.array([0.1, 0.3, 0.5, 0.7, 0.9])).sum(axis=1)

assert(kmeans_centroids.shape[0] == 100) # This should be 100
kmeans = KMeans(kmeans_centroids.shape[0])
kmeans.cluster_centers_ = kmeans_centroids
kmeans._n_threads = -1

raw_data_classes = kmeans.predict(raw_data)

pca_3d = PCA(n_components=3) # 3D vizualization
PCs_3d = pca_3d.fit_transform(raw_data)
PCs_3d_centroids = pca_3d.fit_transform(kmeans_centroids)

df = pd.DataFrame(PCs_3d, columns=['dim1', 'dim2', 'dim3'])
df_centoids = pd.DataFrame(PCs_3d_centroids, columns=['dim1', 'dim2', 'dim3'])

marker_text = []
for i, card_string in enumerate(cards):
	cards = card_string.split(' ')
	assert(len(cards) == 6)
	marker_text.append(f'Player Cards: {cards[:2]}, Community Cards: {cards[2:]}, Class: {raw_data_classes[i]}, EHS: {ehs_raw_data[i]:.2f}')

# # https://stackoverflow.com/questions/61227248/plotly-how-to-create-custom-hover-labels-for-plotly-3d-scatter-figures
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=PCs_3d[:, 0], y=PCs_3d[:,1], z=PCs_3d[:,2],
                        hovertemplate='<b>%{text}</b><extra></extra>',
                        text=marker_text,
                        marker=dict(
                            size=2,
                            color=raw_data_classes
                        ),
                        mode='markers'
))

fig.update_layout(title=dict(text=f"Turn Abstraction ({kmeans_centroids.shape[0]} Clusters)", font=dict(size=28)), title_x=0.5, scene=dict(
                  xaxis_title="Dimension 1",
                  yaxis_title="Dimension 2",
                  zaxis_title="Dimension 3")
)
# fig.show()
fig.show(renderer='browser')

# River Visualization

In [38]:
kmeans_centroids = joblib.load('../data/clusters/river/1669110228_samples=100000_bins=5.npy')
raw_data = np.load('../data/raw/river/1669110228_samples=100000_bins=5.npy')
cards = joblib.load('../data/raw/river/1669110228_samples=100000_bins=5')

# Convert histograms to EHS
ehs_raw_data = (raw_data * np.array([0.1, 0.3, 0.5, 0.7, 0.9])).sum(axis=1)

assert(kmeans_centroids.shape[0] == 100) # This should be 100
kmeans = KMeans(kmeans_centroids.shape[0])
kmeans.cluster_centers_ = kmeans_centroids
kmeans._n_threads = -1

raw_data_classes = kmeans.predict(raw_data)

pca_3d = PCA(n_components=3) # 3D vizualization
PCs_3d = pca_3d.fit_transform(raw_data)
PCs_3d_centroids = pca_3d.fit_transform(kmeans_centroids)

df = pd.DataFrame(PCs_3d, columns=['dim1', 'dim2', 'dim3'])
df_centoids = pd.DataFrame(PCs_3d_centroids, columns=['dim1', 'dim2', 'dim3'])

marker_text = []
for i, card_string in enumerate(cards):
	cards = card_string.split(' ')
	assert(len(cards) == 7)
	marker_text.append(f'Player Cards: {cards[:2]}, Community Cards: {cards[2:]}, Class: {raw_data_classes[i]}, EHS: {ehs_raw_data[i]:.2f}')

# # https://stackoverflow.com/questions/61227248/plotly-how-to-create-custom-hover-labels-for-plotly-3d-scatter-figures
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=PCs_3d[:, 0], y=PCs_3d[:,1], z=PCs_3d[:,2],
                        hovertemplate='<b>%{text}</b><extra></extra>',
                        text=marker_text,
                        marker=dict(
                            size=2,
                            color=raw_data_classes
                        ),
                        mode='markers'
))

fig.update_layout(title=dict(text=f"River Abstraction ({kmeans_centroids.shape[0]} Clusters)", font=dict(size=28)), title_x=0.5, scene=dict(
                  xaxis_title="Dimension 1",
                  yaxis_title="Dimension 2",
                  zaxis_title="Dimension 3")
)
# fig.show()

fig.show(renderer='browser')